In [ ]:
from flask import Flask, render_template, request
import pickle
import pandas as pd

app = Flask(__name__)

# Load trained model AND feature names
with open("churn_model.pkl", "rb") as f:
    model, feature_names = pickle.load(f)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # 1. Get raw input values
        tenure_raw = request.form.get("tenure", "").strip()
        monthly_raw = request.form.get("MonthlyCharges", "").strip()
        total_raw = request.form.get("TotalCharges", "").strip()
        internet_raw = request.form.get("InternetService", "").strip()
        contract_raw = request.form.get("Contract", "").strip()

        # 2. Check for missing fields
        if not all([tenure_raw, monthly_raw, total_raw, internet_raw, contract_raw]):
            raise ValueError("All fields are required. Please fill in all inputs.")

        # 3. Convert to correct data types
        tenure = int(tenure_raw)
        monthly = float(monthly_raw)
        total = float(total_raw)
        internet = int(internet_raw)
        contract = int(contract_raw)

        # 4. Validate input values
        if tenure < 0 or monthly < 0 or total < 0:
            raise ValueError("Values cannot be negative.")

        if internet not in [0, 1, 2]:
            raise ValueError("Internet Service must be 0, 1, or 2.")

        if contract not in [0, 1, 2]:
            raise ValueError("Contract type must be 0, 1, or 2.")

        # 5. Create input DataFrame (SAFE)
        input_df = pd.DataFrame(0, columns=feature_names, index=[0])

        input_df.at[0, "tenure"] = tenure
        input_df.at[0, "MonthlyCharges"] = monthly
        input_df.at[0, "TotalCharges"] = total
        input_df.at[0, "InternetService"] = internet
        input_df.at[0, "Contract"] = contract

        # 6. Predict churn
        prediction = model.predict(input_df)[0]

        result = (
            "Customer Will Churn ❌"
            if prediction == 1
            else "Customer Will Stay ✅"
        )

        return render_template("index.html", prediction=result)

    except ValueError as e:
        return render_template(
            "index.html",
            validation_message=str(e)
        )

    except Exception:
        return render_template(
            "index.html",
            validation_message="An unexpected error occurred. Please try again."
        )

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
